In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from config import model_config, hp
from AE.model.autoencoder import Autoencoder
from train import train


In [2]:
model = Autoencoder(model_config)
trained_model, loss = train(model, 'data/complex_noisy_data.pkl')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: carlosmari (networkrl). Use `wandb login --relogin` to force relogin


  3%|▎         | 7/250 [00:15<08:45,  2.16s/it]